# Bounding box drawer
This notebook will be used to test the bounding boxes in the iNaturalist2017 dataset

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import json

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/andrei/.local/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/andrei/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/andrei/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/andrei/.local/lib/python2.7/site-p

# Model preparation 

## Paths

Select the paths to the json file, output folder

In [4]:
# Path to the json file
PATH_TO_JSON = "/home/andrei/workspace/animal-recognition/json_parser/mammal_val2017.json"
PATH_TO_LABELS = "/home/andrei/workspace/animal-recognition/tfrecord_converter/mammal_label_map.pbtxt"
NUM_CLASSES = 186

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [12]:
json_data = json.load(open(PATH_TO_JSON))
index = 0
for example in json_data:
    try:
        image = Image.open(example["file_name"])
        #image_np = load_image_into_numpy_array(image)
        for bbox, category in zip(example["bbox"], example["category_name"]):
            vis_util.draw_bounding_box_on_image(
                    image,
                    bbox[0],
                    bbox[1],
                    bbox[2],
                    bbox[3],
                    color='green',
                    display_str_list=[category])
        #plt.imshow(image_np)
        image.save("val_2017_images/" + str(index) + ".jpg")
        index += 1
    except:
        continue
'''for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  print (output_dict)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)'''

"for image_path in TEST_IMAGE_PATHS:\n  image = Image.open(image_path)\n  # the array based representation of the image will be used later in order to prepare the\n  # result image with boxes and labels on it.\n  image_np = load_image_into_numpy_array(image)\n  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]\n  image_np_expanded = np.expand_dims(image_np, axis=0)\n  # Actual detection.\n  output_dict = run_inference_for_single_image(image_np, detection_graph)\n  print (output_dict)\n  # Visualization of the results of a detection.\n  vis_util.visualize_boxes_and_labels_on_image_array(\n      image_np,\n      output_dict['detection_boxes'],\n      output_dict['detection_classes'],\n      output_dict['detection_scores'],\n      category_index,\n      instance_masks=output_dict.get('detection_masks'),\n      use_normalized_coordinates=True,\n      line_thickness=8)\n  plt.figure(figsize=IMAGE_SIZE)\n  plt.imshow(image_np)"